# vaccine-doses-on-hand

By [Sean Greene](https://www.latimes.com/people/sean-greene)

Downloads the number of vaccine doses on hand from a Tableau dashboard published by the California Department of Public Health.

## Import

Code formatting with [black](https://pypi.org/project/nb-black/).

Import dependencies.

In [2]:
import os
import pytz
from datetime import datetime

In [3]:
from tableauscraper import TableauScraper as TS

## Scrape

Set the URL

In [4]:
host = "https://public.tableau.com"

In [5]:
path = "/views/COVID-19VaccineProviderDashboardPublic/PublicVaccineProviderDashboard"

In [6]:
url = f"{host}{path}"

In [7]:
ts = TS()
ts.loads(url)

In [11]:
dashboard = ts.getWorkbook()

In [12]:
target = "Main Vaccine Data Check (6)"

In [13]:
sheet = next(w for w in dashboard.worksheets if w.name == target)

In [14]:
df = sheet.data

In [12]:
df

,MCE/LHJ/LTCF-value,MCE/LHJ/LTCF-alias,Type-value,Type-alias,AGG(Total On Hand Tiberius)-alias
0,Adventist Health,Adventist Health,MCE,Multi-County Entity,30867
1,AHMC,AHMC,MCE,Multi-County Entity,5177
2,Alameda County Public Health Department,Alameda County Public Health Department,LHJ,Local Health Jurisdiction,91881
3,Albertsons,Albertsons,LTCF,LTCF,210655
4,Alpine County Health and Human Services,Alpine County Health and Human Services,LHJ,Local Health Jurisdiction,690
...,...,...,...,...,...
83,UC Health,UC Health,MCE,Multi-County Entity,88194
84,Ventura County Health Department,Ventura County Health Department,LHJ,Local Health Jurisdiction,28019
85,Walgreens,Walgreens,LTCF,LTCF,176692
86,Walmart,Walmart,LTCF,LTCF,6207


## Export

Save out the data as a CSV that's datestamped to California time.

In [ ]:
tz = pytz.timezone("America/Los_Angeles")

In [ ]:
today = datetime.now(tz).date()

In [ ]:
data_dir = os.path.join(os.path.abspath(""), "data")

In [ ]:
df.to_csv(os.path.join(data_dir, f"{today}.csv"), index=False)